In [1]:
# @title Mounting Google Drive
import os
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/'
HOME = os.getcwd()

KeyboardInterrupt: 

In [ ]:
#@title Load database
import pandas as pd
data_path = '/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/dataframeAcumenFinal.csv'
df_acumen = pd.read_csv(data_path)
#df_acumen.info()
df_acumen['manufacturer2'].unique()
#df_acumen['category'].unique()

array(['Shelf', 'Desk', 'Stool', 'Bench', 'Table', 'Lamp', 'Chair',
       'Board', 'Mirror'], dtype=object)

In [ ]:
# @title importing supervision library
!pip install -q supervision
import supervision as sv

In [ ]:
# @title loading Grounding DINO
#!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .


CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
%cd {HOME}/weights
#!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
%cd {HOME}/GroundingDINO


from groundingdino.util.inference import load_model, load_image, predict, annotate
GDINO = load_model(CONFIG_PATH, WEIGHTS_PATH)
%cd {HOME}

/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/GroundingDINO
  Preparing metadata (setup.py) ... done
/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/weights
/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/GroundingDINO


/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/GroundingDINO/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased
/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification


In [ ]:
# @title loading CLIP
from transformers import CLIPProcessor, CLIPImageProcessor, CLIPModel, CLIPPreTrainedModel
model_ID = "openai/clip-vit-base-patch32"
ICLIP = CLIPModel.from_pretrained(model_ID)
preprocess = CLIPImageProcessor.from_pretrained(model_ID)

In [ ]:
# @title necessary libraries
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from PIL import Image
import os
import requests
import io
import numpy as np
from torchvision import transforms
import pandas as pd
from io import BytesIO
import re
import json

In [ ]:
# @title OD(image_source, text_prompt, threshold)
def OD(image_source, text_prompt, threshold):
    normalized_array = image_source / 255.0
    transposed_array = normalized_array.transpose((2, 0, 1))
    image_tensor = torch.tensor(transposed_array, dtype=torch.float32)

    boxes, logits, phrases = predict(
        device='cpu',
        model=GDINO,
        image=image_tensor,
        caption=text_prompt,
        box_threshold=threshold,
        text_threshold=threshold
    )

    unique_phrases = set(phrases)  # Get unique phrases

    filtered_boxes = []
    filtered_logits = []
    filtered_phrases = []

    for phrase in unique_phrases:
        # Find indices where the phrase occurs
        indices = [i for i, p in enumerate(phrases) if p == phrase]

        # Find the index with the highest logits
        max_index = max(indices, key=lambda i: logits[i])

        # Add the item with the highest logits to the filtered lists
        filtered_boxes.append(boxes[max_index])
        filtered_logits.append(logits[max_index])
        filtered_phrases.append(phrases[max_index])

    return filtered_boxes, filtered_logits, filtered_phrases

In [ ]:
# @title plot_boxes(image_url, text_prompt_list, threshold)
def plot_boxes(image_url, text_prompt_list, threshold):
    image_array = url_to_image_array(image_url)
    image = Image.fromarray(image_array)
    img_width, img_height = image.size

    fig, ax = plt.subplots(1, figsize=(16, 12))
    ax.set_axis_off()

    ax.imshow(image)

    for text_prompt in text_prompt_list:
        boxes, logits, phrases = OD(image_array, text_prompt, threshold)

        for i, box in enumerate(boxes):
            x_center, y_center, width, height = box.tolist()
            x_min = max(0, (x_center - width / 2) * img_width)
            y_min = max(0, (y_center - height / 2) * img_height)
            x_max = min(img_width, (x_center + width / 2) * img_width)
            y_max = min(img_height, (y_center + height / 2) * img_height)

            rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=3, edgecolor='r', facecolor='none')

            ax.add_patch(rect)

            phrase = phrases[i]
            logit = logits[i]
            logit_percentage = int(logits[i] * 100)
            ax.text(x_min, y_min - 5, f"{phrase} ({logit_percentage:.0f}%)", color='r', fontsize=15, ha='left', va='bottom')

    plt.show()

In [ ]:
# @title plot_dots(image_url, text_prompt_list, threshold)
def plot_dots(image_url, text_prompt_list, threshold):
    image_array = url_to_image_array(image_url)
    image = Image.fromarray(image_array)
    img_width, img_height = image.size

    fig, ax = plt.subplots(1, figsize=(16, 12))
    ax.set_axis_off()

    ax.imshow(image)

    for text_prompt in text_prompt_list:
        boxes, logits, phrases = OD(image_array, text_prompt, threshold)
        for i, box in enumerate(boxes):
            x_center, y_center, width, height = box.tolist()
            x_min = max(0, (x_center - width / 2) * img_width)
            y_min = max(0, (y_center - height / 2) * img_height)
            x_max = min(img_width, (x_center + width / 2) * img_width)
            y_max = min(img_height, (y_center + height / 2) * img_height)
            x = x_center * img_width
            y = y_center * img_height

            dot_x = x
            dot_y = y_min

            ax.plot(dot_x, dot_y, marker='o', markersize=12, color='red', label='Dot')

            phrase = phrases[i]
            logit = logits[i]
            logit_percentage = int(logits[i] * 100)
            ax.text(dot_x - 50, dot_y + 10, f"{phrase} ({logit_percentage:.0f}%)", color='red', fontsize=15)

    plt.show()

In [ ]:
# @title get_ROI_images(image_source, text_prompt, threshold)
def get_ROI_images(image_source, text_prompt, threshold):
    detected_objects = []
    image = Image.fromarray(image_source).convert('RGB')
    img_width, img_height = image.size
    bounding_boxes, accuracies, labels = OD(image_source, text_prompt, threshold)

    for i, box in enumerate(bounding_boxes):
        x_center, y_center, width, height = box.tolist()
        x_min = max(0, (x_center - width / 2) * img_width)
        y_min = max(0, (y_center - height / 2) * img_height)
        x_max = min(img_width, (x_center + width / 2) * img_width)
        y_max = min(img_height, (y_center + height / 2) * img_height)
        detected_object = image.crop((x_min, y_min, x_max, y_max))
        detected_object = detected_object.resize((img_width, img_height), Image.ANTIALIAS)
        img_array = np.array(detected_object)
        detected_objects.append(img_array)

    return detected_objects, labels

In [ ]:
#@title get_ROI_images_and_boxes(image_source, text_prompt, threshold)
def get_ROI_images_and_boxes(image_source, text_prompt, threshold):
    detected_objects = []
    image = Image.fromarray(image_source).convert('RGB')
    img_width, img_height = image.size
    bounding_boxes, accuracies, labels = OD(image_source, text_prompt, threshold)

    for i, box in enumerate(bounding_boxes):
        x_center, y_center, width, height = box.tolist()
        x_min = max(0, (x_center - width / 2) * img_width)
        y_min = max(0, (y_center - height / 2) * img_height)
        x_max = min(img_width, (x_center + width / 2) * img_width)
        y_max = min(img_height, (y_center + height / 2) * img_height)
        x = x_center * img_width
        y = y_min
        width_ = width * img_width
        height_ = height * img_height
        detected_object = image.crop((x_min, y_min, x_max, y_max))
        detected_object = detected_object.resize((img_width, img_height), Image.ANTIALIAS)
        img_array = np.array(detected_object)
        detected_objects.append((img_array, (x, y, width_, height_)))

    return detected_objects, labels

In [ ]:
# @title image_load_and_preprocess(image)
def image_load_and_preprocess(image):
    image = image.convert("RGB")
    image = preprocess(image, return_tensors="pt")
    return image

def path_load_and_preprocess(image_path):
    image = Image.open(image_path)
    image = image.convert("RGB")
    image = preprocess(image, return_tensors="pt")
    return image

In [ ]:
# @title convert_numpy_to_image(numpy_images)
def convert_numpy_to_image(numpy_images):
    image_objects = []

    for numpy_img in numpy_images:
        image_obj = Image.fromarray(np.uint8(numpy_img))
        image_objects.append(image_obj)

    return image_objects

In [ ]:
# @title check_labels(filename, label_list)
def check_labels(filename, label_list):
    cleaned_name = re.sub(r'[^a-zA-Z\s]', '', filename)
    combined_name = ' '.join(cleaned_name.split())
    combined_labels = ' '.join(label_list)

    words_set_1 = set(combined_name.lower().split())
    words_set_2 = set(combined_labels.lower().split())

    matched_words = words_set_1.intersection(words_set_2)
    #if len(matched_words)>0:
      #print(f"Matched words: {', '.join(matched_words)}")
    return any(word in words_set_2 for word in words_set_1)


def check_labels_v2(filename, label_list):
    return [label for label in label_list if filename.lower() == label.lower()]

In [ ]:
# @title url_to_image_array(url)
def url_to_image_array(url):

    response = requests.get(url)
    image_ac = Image.open(BytesIO(response.content))
    image_ac = image_ac.convert('RGB')
    image_array = np.array(image_ac)

    return image_array

In [ ]:
#@title transform_image(image)
def transform_image(image):
    aspect_ratio = image.width / image.height

    if image.width > image.height:
        new_width = 240
        new_height = int(120 / aspect_ratio)
    else:
        new_width = int(240 * aspect_ratio)
        new_height = 120

    transform = transforms.Compose([
        transforms.Resize((new_height, new_width)),
        transforms.CenterCrop(240)
    ])

    return transform(image)

In [ ]:
#@title process_images_and_get_scores(url, text_prompt_list, dataframe, threshold)
def process_images_and_get_scores(url, text_prompt_list, dataframe, threshold):
    similarity_model = ICLIP

    image_array = url_to_image_array(url)

    images_a_arrays_list = []
    labels_list = []
    similarity_scores_list_list = []
    id_values_list_list = []

    for prompt in text_prompt_list:
        images_a_arrays, labels = get_ROI_images(image_source=image_array, text_prompt=prompt, threshold=threshold)

        image_objects = convert_numpy_to_image(images_a_arrays)
        images_a = [im for im in image_objects]
        c = 0
        similarity_scores_list = []
        id_values_list = []

        for index_l, image_a in enumerate(images_a):
            image_a = transform_image(image_a)
            image_a = image_load_and_preprocess(image_a)["pixel_values"]

            with torch.no_grad():
                embedding_a = similarity_model.get_image_features(image_a)
            embedding_a = torch.nn.functional.normalize(embedding_a, p=2, dim=1)

            similarity_scores = []

            for idx, image_b_url in enumerate(dataframe['image']):
                if check_labels(dataframe.loc[idx, 'manufacturer2'], labels):
                    c += 1
                    response = requests.get(image_b_url)
                    image_b_source = Image.open(BytesIO(response.content))
                    image_b = image_b_source.convert('RGB')

                    image_b = transform_image(image_b)
                    image_b = image_load_and_preprocess(image_b)["pixel_values"]

                    with torch.no_grad():
                        embedding_b = similarity_model.get_image_features(image_b)
                    embedding_b = torch.nn.functional.normalize(embedding_b, p=2, dim=1)

                    similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)

                    similarity_scores.append((similarity_score.item(), image_b_source, dataframe.loc[idx, 'id']))

            similarity_scores.sort(reverse=True, key=lambda x: x[0])
            similarity_scores = [score_tuple for score_tuple in similarity_scores if score_tuple[0] >= 0.75]
            similarity_scores = similarity_scores[:4]
            similarity_scores_list.append(similarity_scores)

        print(f'Number of fetched rows for prompt "{prompt}": {c}')
        images_a_arrays_list.append(images_a_arrays)
        labels_list.append(labels)
        similarity_scores_list_list.append(similarity_scores_list)

    return images_a_arrays_list, labels_list, similarity_scores_list_list

In [ ]:
# @title plot_similarity_scores(images_a_arrays_list, labels_list, similarity_scores_list_list, text_prompt_list)
def plot_similarity_scores(images_a_arrays_list, labels_list, similarity_scores_list_list, text_prompt_list):
    for prompt, images_a_arrays, labels, similarity_scores_list in zip(text_prompt_list, images_a_arrays_list, labels_list, similarity_scores_list_list):

        for idx, (image_a, similarity_scores) in enumerate(zip(images_a_arrays, similarity_scores_list)):
            num_images_to_plot = min(6, len(similarity_scores))

            fig_input, axes_input = plt.subplots(1, 7, figsize=(20, 3))
            fig_input.suptitle(f'Input Image {labels[idx]} ({prompt})')
            img_input = image_a
            axes_input[0].imshow(img_input)
            axes_input[0].set_title('Input Image')
            axes_input[0].axis('off')

            for i in range(num_images_to_plot):
                img = similarity_scores[i][1]
                axes_input[i + 1].imshow(img)
                axes_input[i + 1].set_title(f"Similarity: {similarity_scores[i][0] * 100:.0f}%\nID: {similarity_scores[i][2]}")
                axes_input[i + 1].axis('off')

            plt.tight_layout()
            plt.show()

In [ ]:
# @title get_outputs(url, text_prompt_list, dataframe, detection_model, similarity_model, threshold)
def get_outputs(url, text_prompt_list, dataframe, threshold):
    similarity_model = ICLIP

    image_array = url_to_image_array(url)

    similarity_list = []

    for prompt in text_prompt_list:
        detected_objects, labels = get_ROI_images_and_boxes(image_source=image_array, text_prompt=prompt, threshold=threshold)

        for index_l, (image_a_array, bounding_box) in enumerate(detected_objects):
            x, y, width_, height_ = bounding_box


            image_obj = Image.fromarray(np.uint8(image_a_array))
            image_obj = transform_image(image_obj)
            image_a = image_load_and_preprocess(image_obj)["pixel_values"]
            with torch.no_grad():
                embedding_a = similarity_model.get_image_features(image_a)
            embedding_a = torch.nn.functional.normalize(embedding_a, p=2, dim=1)

            similarity_scores = []

            for idx, image_b_url in enumerate(dataframe['image']):
                if check_labels(dataframe.loc[idx, 'manufacturer2'], labels):
                    response = requests.get(image_b_url)
                    image_b_source = Image.open(BytesIO(response.content))
                    image_b_source = image_b_source.convert('RGB')

                    image_b_source = transform_image(image_b_source)
                    image_b_array = image_load_and_preprocess(image_b_source)["pixel_values"]

                    with torch.no_grad():
                        embedding_b = similarity_model.get_image_features(torch.tensor(image_b_array, dtype=torch.float32))
                    embedding_b = torch.nn.functional.normalize(embedding_b, p=2, dim=1)

                    similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)

                    similarity_scores.append((similarity_score.item(), image_b_source, dataframe.loc[idx, 'id']))

            similarity_scores.sort(reverse=True, key=lambda x: x[0])
            similarity_scores = [score_tuple for score_tuple in similarity_scores if score_tuple[0] >= 0.80]

            similarity_scores = similarity_scores[:4]
            product_matches = []

            for similarity_score in similarity_scores:
                confidence, image_b_source, product_id = similarity_score

                product_match = {
                    "product_id": str(product_id),
                    "confidence": str(confidence),
                    "x": str(x),
                    "y": str(y),
                    "width": str(width_),
                    "height": str(height_)
                }

                product_matches.append(product_match)

            similarity_list.append({
                'prompt': prompt,
                'images_a_array': image_a_array,
                'labels': labels,
                'similarity_scores': product_matches
            })

    return url, image_array.shape[1], image_array.shape[0], similarity_list

In [ ]:
# @title get_json_output(url, img_width, img_height, similarity_list)
def get_json_output(url, img_width, img_height, similarity_list):
    json_structure = []

    for similarity_item in similarity_list:
        prompt = similarity_item['prompt']
        image_a_array = similarity_item['images_a_array']
        labels = similarity_item['labels']
        similarity_scores = similarity_item['similarity_scores']

        product_matches = []

        for similarity_score in similarity_scores:
            product_id = similarity_score['product_id']
            confidence = similarity_score['confidence']
            x, y, width, height = map(float, [similarity_score['x'], similarity_score['y'], similarity_score['width'], similarity_score['height']])

            product_match = {
                "product_id": str(product_id),
                "confidence": str(confidence),
                "x": str(x),
                "y": str(y),
                "width": str(width),
                "height": str(height)
            }

            product_matches.append(product_match)

        json_item = {
            "source_media": {
                "media_url": url,
                "width": str(img_width),
                "height": str(img_height)
            },
            "product_matches": product_matches,
            "tags": [],
            "errors": []
        }

        json_structure.append(json_item)

    return json_structure

In [ ]:
#@title output_json(url)
def output_json(url):
    threshold = 0.5
    data_path = '/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/dataframeAcumenFinal.csv'
    dataframe = pd.read_csv(data_path)
    text_prompt_list = ['chair',  'stool', 'table', 'shelf', 'desk', 'lamp', 'board', 'mirrior', 'bench']
    media_url, img_width, img_height, similarity_list = get_outputs(url, text_prompt_list, dataframe, threshold=threshold)
    output = get_json_output(media_url, img_width, img_height, similarity_list)
    return output

In [ ]:
#@title output_plot(url)
def output_plot(url):
    threshold = 0.5
    data_path = '/content/drive/MyDrive/Capstone Project/Acumen_OD_Classification/dataframeAcumenFinal.csv'
    dataframe = pd.read_csv(data_path)
    text_prompt_list = ['chair',  'stool', 'table', 'shelf', 'desk', 'lamp', 'board', 'mirrior', 'bench']
    images_a_arrays_list, labels_list, similarity_scores_list_list = process_images_and_get_scores(url, text_prompt_list, dataframe, threshold)
    plot_similarity_scores(images_a_arrays_list, labels_list, similarity_scores_list_list, text_prompt_list)

In [ ]:
urls = [
    'https://app.oculizm.com/wp-content/uploads/2023/04/insta_profile_83953-155183964_471925360844077_6321490016420926228_n.jpg',
    'https://app.oculizm.com/wp-content/uploads/2023/04/insta_profile_83953-116957702_181330093377725_7625150121191732645_n-1024x976.jpg',
    'https://app.oculizm.com/wp-content/uploads/2023/04/insta_profile_83953-118980998_328984211683835_1353543039422003611_n-1024x1024.jpg',
    'https://app.oculizm.com/wp-content/uploads/2023/04/insta_profile_83953-104433654_119498016181098_7689914167204045686_n.jpg'
]

In [ ]:
#output_plot(urls[0])
#output = output_json(urls[0])
#print(json.dumps(output, indent=2))

In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.2 M

In [ ]:
import gradio as gr

In [ ]:
#@title process_images_gradio(image_input)
import numpy as np
def process_images_gradio(input_image):
    text_prompt_list = ['chair',  'stool', 'table', 'shelf', 'desk', 'lamp', 'board', 'mirrior', 'bench']
    dataframe = df_acumen
    threshold = 0.6
    similarity_model = ICLIP

    image_array = np.array(input_image)

    images_a_arrays_list = []
    labels_list = []
    similarity_scores_list_list = []
    id_values_list_list = []
    outputs_images_list = []

    for prompt in text_prompt_list:
        images_a_arrays, labels = get_ROI_images(image_source=image_array, text_prompt=prompt, threshold=threshold)

        image_objects = convert_numpy_to_image(images_a_arrays)
        images_a = [im for im in image_objects]
        similarity_scores_list = []
        id_values_list = []

        for index_l, image_a in enumerate(images_a):
            image_a = transform_image(image_a)
            image_a = image_load_and_preprocess(image_a)["pixel_values"]

            with torch.no_grad():
                embedding_a = similarity_model.get_image_features(image_a)
            embedding_a = torch.nn.functional.normalize(embedding_a, p=2, dim=1)

            similarity_scores = []

            for idx, image_b_url in enumerate(dataframe['image']):
                if check_labels(dataframe.loc[idx, 'manufacturer2'], labels):
                    response = requests.get(image_b_url)
                    image_b_source = Image.open(BytesIO(response.content))
                    image_b = image_b_source.convert('RGB')

                    image_b = transform_image(image_b)
                    image_b = image_load_and_preprocess(image_b)["pixel_values"]

                    with torch.no_grad():
                        embedding_b = similarity_model.get_image_features(image_b)
                    embedding_b = torch.nn.functional.normalize(embedding_b, p=2, dim=1)

                    similarity_score = torch.nn.functional.cosine_similarity(embedding_a, embedding_b)

                    similarity_scores.append((similarity_score.item(), image_b_source, dataframe.loc[idx, 'id']))
                    similarity_scores = [score_tuple for score_tuple in similarity_scores if score_tuple[0] >= 0.4]

            similarity_scores.sort(reverse=True, key=lambda x: x[0])
            similarity_scores = similarity_scores[:4]
            similarity_scores_list.append(similarity_scores)
            outputs_images_list.append([(np.array(score_tuple[1]))/255.0 for score_tuple in similarity_scores])


        images_a_arrays_list.append(images_a_arrays)
        labels_list.append(labels)
        similarity_scores_list_list.append(similarity_scores_list)


    return outputs_images_list[0]

In [ ]:
demo = gr.Interface(fn=process_images_gradio, inputs=gr.Image(width=600, height=400), outputs=[gr.Image(width=600, height=400), gr.Image(width=600, height=400), gr.Image(width=600, height=400), gr.Image(width=600, height=400)])
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://aa97edb88f40281745.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
